### Using SciKit Learn

In [1]:
from sklearn.neural_network import MLPRegressor
import pandas as pd
from matplotlib import pyplot as plt 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error , mean_squared_error, mean_absolute_error 
import numpy as np
from datetime import datetime

### Reading the Dataset

In [2]:
dataset = pd.read_csv("cleaneddata.csv")
df = dataset

In [3]:
dataset.head()

,Date Created,Solar Voltage,Solar Current,Battery Voltage,Inverter Charge Current,Usage Current,Solar Harvest,Battery Charge,Inverter Charge,Usage Charge,Date Extended,Unnamed: 11,Unnamed: 12
0,04/01/2022 12:08,15.050969,0.920469,13.577745,0,-5.889368,15.189676,-∞,2.400000e+17,∞,,NaN,NaN
1,04/01/2022 12:08,15.050969,0.980130,13.586268,0,-6.008690,15.193615,-∞,2.400000e+17,∞,,NaN,NaN
2,04/01/2022 12:08,15.085061,1.005699,13.611837,0,-6.051305,15.197501,-∞,2.400000e+17,∞,,NaN,NaN
3,04/01/2022 12:09,15.264044,1.150590,11.915763,0,-6.281425,15.202242,-∞,2.400000e+17,∞,,NaN,NaN
4,04/01/2022 12:09,15.494164,0.264199,13.969803,0,-6.170627,15.207151,-∞,2.400000e+17,∞,,NaN,NaN


### Cleaning Dataset Before Using It For Modelling

In [4]:
# To get the list of columns and their data types  
df.dtypes

Date Created                 object
 Solar Voltage              float64
 Solar Current              float64
 Battery Voltage            float64
 Inverter Charge Current      int64
 Usage Current              float64
 Solar Harvest              float64
 Battery Charge              object
 Inverter Charge            float64
 Usage Charge                object
 Date Extended               object
Unnamed: 11                 float64
Unnamed: 12                 float64
dtype: object

In [5]:
df.describe()

,Solar Voltage,Solar Current,Battery Voltage,Inverter Charge Current,Usage Current,Solar Harvest,Inverter Charge,Unnamed: 11,Unnamed: 12
count,702.000000,702.000000,702.000000,702.0,702.000000,702.000000,7.020000e+02,0.0,0.0
mean,13.828819,0.948759,12.331107,0.0,-4.721148,22.225146,6.758632e+17,NaN,NaN
std,0.439597,0.611725,0.454373,0.0,2.099007,22.162818,3.595264e+17,NaN,NaN
min,13.227050,0.085217,10.415717,0.0,-7.542828,3.389207,2.400000e+17,NaN,NaN
25%,13.593538,0.528412,12.094746,0.0,-6.140796,4.063559,2.400000e+17,NaN,NaN
50%,13.661722,0.732964,12.162930,0.0,-5.633678,20.331717,9.720000e+17,NaN,NaN
75%,13.983465,1.404149,12.495326,0.0,-3.426225,29.218991,9.720000e+17,NaN,NaN
max,16.346462,2.573929,14.881763,0.0,-0.801145,69.312950,9.720000e+17,NaN,NaN


In [6]:
#Renaming the Columns to include an underscore instead of space

df.rename(columns={' Battery Voltage':'Battery_Voltage',' Solar Voltage':'Solar_Voltage',' Solar Current':'Solar_Current', 'Date Created':'Date_Created',' Usage Current':'Usage_Current','Unnamed: 11':'Battery_Current','Unnamed: 12':'Solar_Power' ,' Date Extended':'Date_Extended' }, inplace=True)
df.columns

Index(['Date_Created', 'Solar_Voltage', 'Solar_Current', 'Battery_Voltage',
       ' Inverter Charge Current', 'Usage_Current', ' Solar Harvest',
       ' Battery Charge', ' Inverter Charge', ' Usage Charge', 'Date_Extended',
       'Battery_Current', 'Solar_Power'],
      dtype='object')

In [28]:
# Creating new columns for caclulated parameters

df['Battery_Current']=  df['Solar_Current'] + (df['Usage_Current']/2)
df['Solar_Power'] = df['Solar_Voltage'] * df['Solar_Current']
df['load_Power'] = df['Battery_Voltage'] * df['Usage_Current']/2 * -1
df['Battery_Power'] = df['Battery_Voltage'] * df['Battery_Current'] * -1
df['Total_Dc_Power'] = df['Solar_Power'] + df['Battery_Power']
df['Efficiency'] = (df['load_Power']/df['Total_Dc_Power']) * 100

In [8]:
df.head()

,Date_Created,Solar_Voltage,Solar_Current,Battery_Voltage,Inverter Charge Current,Usage_Current,Solar Harvest,Battery Charge,Inverter Charge,Usage Charge,Date_Extended,Battery_Current,Solar_Power,load_Power,Battery_Power,Total_Dc_Power,Efficiency
0,04/01/2022 12:08,15.050969,0.920469,13.577745,0,-5.889368,15.189676,-∞,2.400000e+17,∞,,-2.024215,13.853956,39.982170,27.484272,41.338227,96.719605
1,04/01/2022 12:08,15.050969,0.980130,13.586268,0,-6.008690,15.193615,-∞,2.400000e+17,∞,,-2.024215,14.751910,40.817837,27.501524,42.253434,96.602412
2,04/01/2022 12:08,15.085061,1.005699,13.611837,0,-6.051305,15.197501,-∞,2.400000e+17,∞,,-2.019953,15.171035,41.184689,27.495274,42.666309,96.527424
3,04/01/2022 12:09,15.264044,1.150590,11.915763,0,-6.281425,15.202242,-∞,2.400000e+17,∞,,-1.990123,17.562657,37.423988,23.713830,41.276487,90.666602
4,04/01/2022 12:09,15.494164,0.264199,13.969803,0,-6.170627,15.207151,-∞,2.400000e+17,∞,,-2.821114,4.093548,43.101219,39.410406,43.503954,99.074256


In [9]:
# Converting the Date Created column from an Object data type to DateTime
df['Date_Created'] = pd.to_datetime(df['Date_Created'])
df['Date_Created']
df.head()

,Date_Created,Solar_Voltage,Solar_Current,Battery_Voltage,Inverter Charge Current,Usage_Current,Solar Harvest,Battery Charge,Inverter Charge,Usage Charge,Date_Extended,Battery_Current,Solar_Power,load_Power,Battery_Power,Total_Dc_Power,Efficiency
0,2022-04-01 12:08:00,15.050969,0.920469,13.577745,0,-5.889368,15.189676,-∞,2.400000e+17,∞,,-2.024215,13.853956,39.982170,27.484272,41.338227,96.719605
1,2022-04-01 12:08:00,15.050969,0.980130,13.586268,0,-6.008690,15.193615,-∞,2.400000e+17,∞,,-2.024215,14.751910,40.817837,27.501524,42.253434,96.602412
2,2022-04-01 12:08:00,15.085061,1.005699,13.611837,0,-6.051305,15.197501,-∞,2.400000e+17,∞,,-2.019953,15.171035,41.184689,27.495274,42.666309,96.527424
3,2022-04-01 12:09:00,15.264044,1.150590,11.915763,0,-6.281425,15.202242,-∞,2.400000e+17,∞,,-1.990123,17.562657,37.423988,23.713830,41.276487,90.666602
4,2022-04-01 12:09:00,15.494164,0.264199,13.969803,0,-6.170627,15.207151,-∞,2.400000e+17,∞,,-2.821114,4.093548,43.101219,39.410406,43.503954,99.074256


In [19]:
#Select rows within a specific time in this case between 7
df[(df['Date_Created'] > '07:30:00') & (df['Date_Created'] < '16:40:00')]
df.tail()

,Date_Created,Solar_Voltage,Solar_Current,Battery_Voltage,Inverter Charge Current,Usage_Current,Solar Harvest,Battery Charge,Inverter Charge,Usage Charge,Date_Extended,Battery_Current,Solar_Power,load_Power,Battery_Power,Total_Dc_Power,Efficiency
697,2022-04-12 16:46:00,13.908889,0.187492,12.452711,0,-1.031265,69.311394,-12.12796879,9.720000e+17,∞,,-0.328140,2.607812,6.421025,4.086236,6.694047,95.921416
698,2022-04-12 16:46:00,13.908889,0.187492,12.452711,0,-1.031265,69.311394,-12.12796879,9.720000e+17,∞,,-0.328140,2.607812,6.421025,4.086236,6.694047,95.921416
699,2022-04-12 16:46:00,13.917412,0.281245,12.452711,0,-1.022742,69.312218,-12.13144302,9.720000e+17,∞,,-0.230126,3.914207,6.367958,2.865691,6.779898,93.924095
700,2022-04-12 16:47:00,13.917412,0.281245,12.452711,0,-1.022742,69.312218,-12.13144302,9.720000e+17,∞,,-0.230126,3.914207,6.367958,2.865691,6.779898,93.924095
701,2022-04-12 16:47:00,13.917412,0.187492,12.452711,0,-1.022742,69.312950,-12.13470554,9.720000e+17,∞,,-0.323879,2.609410,6.367958,4.033169,6.642578,95.865757


In [30]:
#
df=df[['Date_Created',' Inverter Charge Current', 'Usage_Current', ' Solar Harvest',' Battery Charge', ' Inverter Charge', ' Usage Charge', 'Date_Extended','Solar_Voltage', 'Solar_Current', 'Battery_Voltage','Battery_Current', 'Solar_Power','load_Power','Battery_Power', 'Total_Dc_Power', 'Efficiency']]
df.head()

,Date_Created,Inverter Charge Current,Usage_Current,Solar Harvest,Battery Charge,Inverter Charge,Usage Charge,Date_Extended,Solar_Voltage,Solar_Current,Battery_Voltage,Battery_Current,Solar_Power,load_Power,Battery_Power,Total_Dc_Power,Efficiency
0,2022-04-01 12:08:00,0,-5.889368,15.189676,-∞,2.400000e+17,∞,,15.050969,0.920469,13.577745,-2.024215,13.853956,39.982170,27.484272,41.338227,96.719605
1,2022-04-01 12:08:00,0,-6.008690,15.193615,-∞,2.400000e+17,∞,,15.050969,0.980130,13.586268,-2.024215,14.751910,40.817837,27.501524,42.253434,96.602412
2,2022-04-01 12:08:00,0,-6.051305,15.197501,-∞,2.400000e+17,∞,,15.085061,1.005699,13.611837,-2.019953,15.171035,41.184689,27.495274,42.666309,96.527424
3,2022-04-01 12:09:00,0,-6.281425,15.202242,-∞,2.400000e+17,∞,,15.264044,1.150590,11.915763,-1.990123,17.562657,37.423988,23.713830,41.276487,90.666602
4,2022-04-01 12:09:00,0,-6.170627,15.207151,-∞,2.400000e+17,∞,,15.494164,0.264199,13.969803,-2.821114,4.093548,43.101219,39.410406,43.503954,99.074256


In [37]:
# trim the dataset to contain only the parameters that are going in the input layer of the model
train = df.iloc[:,12:15]
train

,Solar_Power,load_Power,Battery_Power
0,13.853956,39.982170,27.484272
1,14.751910,40.817837,27.501524
2,15.171035,41.184689,27.495274
3,17.562657,37.423988,23.713830
4,4.093548,43.101219,39.410406
...,...,...,...
697,2.607812,6.421025,4.086236
698,2.607812,6.421025,4.086236
699,3.914207,6.367958,2.865691
700,3.914207,6.367958,2.865691


In [38]:
# output parameter
test = df['Efficiency']
test

0      96.719605
1      96.602412
2      96.527424
3      90.666602
4      99.074256
         ...    
697    95.921416
698    95.921416
699    93.924095
700    93.924095
701    95.865757
Name: Efficiency, Length: 702, dtype: float64

### Data Normalization

### Input and Output Features Split Into Training and Test splits

In [39]:
X = train #input features
Y = test #output features
Xtrain,Xtest,Ytrain,Ytest = train_test_split(X,Y, random_state=1)

### Instantiate the Neural Network

In [40]:
NN = MLPRegressor(max_iter=500, activation='relu' , hidden_layer_sizes = (100,100))

### Fitting/Training the Network

In [65]:
#training the network
NN.fit(Xtrain,Ytrain)

#predict using test input data
NN_pred = NN.predict(Xtest)

#Compute Errors
print("MSE:" ,  mean_squared_error(Ytest,NN_pred))
print("MAPE:" , mean_absolute_percentage_error(Ytest,NN_pred))
print("MAE:" , mean_absolute_error(Ytest,NN_pred))

MSE: 0.6169187202518779
MAPE: 0.00436114455578602
MAE: 0.40737065897386737
